In [1]:
# JPAULS IPYTHON NOTEBOOK

In [2]:
# Does the necessary imports!
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time

In [3]:
# Gets main page's HTML
mainpagelink = 'http://racingchannel.com/results_archive.php'
mainpagehtml = requests.get(mainpagelink).text

In [4]:
# Gets list of racetracks, racetrackcodes, and creates a dict of the two (can flip the order if necessary)
racetrackshtmllst = BeautifulSoup(mainpagehtml, 'html.parser').body.table.findAll('tr')[4].findAll('td')
tracks = []
trackcodes = []
trackdatalinks = []
for tracklst in racetrackshtmllst:
    tracklinks = tracklst.findAll('a')
    for link in tracklinks:
        #print link.get('href').encode('ascii')
        trackdatalinks.append('http://racingchannel.com/'  + link.get('href').encode('ascii'))
        trackcodes.append(link.get('href')[10:13].encode('ascii'))
        tracks.append(link.text.encode('ascii'))
tracks[1] = 'Arlington Park'
# print tracks
# print '\n\n'
# print trackcodes
# print '\n\n'
trackcodedict = dict(zip(tracks, trackcodes))
tracklinkdict = dict(zip(tracks, trackdatalinks))
# print '\n\n'
trackinfodf = pd.DataFrame(index=tracks)
trackinfodf['code'] = trackcodes
trackinfodf['link'] = trackdatalinks
trackinfodf.head(5)

,code,link
Aqueduct,AQU,http://racingchannel.com//archives/AQU/default...
Arlington Park,ARL,http://racingchannel.com//archives/ARL/default...
Australian Racing,AU1,http://racingchannel.com//archives/AU1/default...
Australian Racing - B,AU2,http://racingchannel.com//archives/AU2/default...
Australian Racing - C,AU3,http://racingchannel.com//archives/AU3/default...


In [122]:
# Stores the html of all track info pages

#############################################################
## SKIP THIS IF TEMPDATA ALREADY HAS TRACKINFO.CSV IN IT!!!! ##
#############################################################


htmloftracks = []
for link in trackdatalinks:
    htmloftracks.append(BeautifulSoup(requests.get(link).text))
    print link
    time.sleep(5)

trackinfodf['track_html'] = htmloftracks
trackinfodf.head(10)

# Stores track codes and links in csv
trackinfodf.to_csv('tempdata/trackinfo.csv', index_label='name')

http://racingchannel.com//archives/AQU/default.html
http://racingchannel.com//archives/ARL/default.html
http://racingchannel.com//archives/AU1/default.html
http://racingchannel.com//archives/AU2/default.html
http://racingchannel.com//archives/AU3/default.html
http://racingchannel.com//archives/BAY/default.html
http://racingchannel.com//archives/BEL/default.html
http://racingchannel.com//archives/BEU/default.html
http://racingchannel.com//archives/CAL/default.html
http://racingchannel.com//archives/CAM/default.html
http://racingchannel.com//archives/CAN/default.html
http://racingchannel.com//archives/CTN/default.html
http://racingchannel.com//archives/CHU/default.html
http://racingchannel.com//archives/COL/default.html
http://racingchannel.com//archives/DDN/default.html
http://racingchannel.com//archives/DEL/default.html
http://racingchannel.com//archives/DPK/default.html
http://racingchannel.com//archives/ELL/default.html
http://racingchannel.com//archives/EMD/default.html
http://racin

,code,link,track_html
Aqueduct,AQU,http://racingchannel.com//archives/AQU/default...,<html> <head><title>Aqueduct Results Archive</...
Arlington Park,ARL,http://racingchannel.com//archives/ARL/default...,<html> <head><title>Arlington Results Archive<...
Australian Racing,AU1,http://racingchannel.com//archives/AU1/default...,<html> <head><title>Australia - A Results Arch...
Australian Racing - B,AU2,http://racingchannel.com//archives/AU2/default...,<html> <head><title>Australia - B Results Arch...
Australian Racing - C,AU3,http://racingchannel.com//archives/AU3/default...,<html> <head><title> Results Archive</title></...
Bay Meadows,BAY,http://racingchannel.com//archives/BAY/default...,<html> <head><title>Bay Meadows Results Archiv...
Belmont Park,BEL,http://racingchannel.com//archives/BEL/default...,<html> <head><title>Belmont Results Archive</t...
Beulah Park,BEU,http://racingchannel.com//archives/BEU/default...,<html> <head><title>Beulah Results Archive</ti...
Calder Race Course,CAL,http://racingchannel.com//archives/CAL/default...,<html> <head><title>Calder Results Archive</ti...
Camarero,CAM,http://racingchannel.com//archives/CAM/default...,<html> <head><title> Results Archive</title></...


In [5]:
# Loads trackinfodf from the csv in tempdata
try:
    del trackinfodf
except:
    pass
trackinfodf = pd.read_csv('tempdata/trackinfo.csv', index_col='name')
trackinfodf['track_html'] = trackinfodf['track_html'].apply(lambda h:BeautifulSoup(h, 'html.parser'))
trackinfodf.head(5)

,code,link,track_html
name,,,
Aqueduct,AQU,http://racingchannel.com//archives/AQU/default...,<html> <head><title>Aqueduct Results Archive</...
Arlington Park,ARL,http://racingchannel.com//archives/ARL/default...,<html> <head><title>Arlington Results Archive<...
Australian Racing,AU1,http://racingchannel.com//archives/AU1/default...,<html> <head><title>Australia - A Results Arch...
Australian Racing - B,AU2,http://racingchannel.com//archives/AU2/default...,<html> <head><title>Australia - B Results Arch...
Australian Racing - C,AU3,http://racingchannel.com//archives/AU3/default...,<html> <head><title> Results Archive</title></...


In [6]:
# For each track rip the years and months that the track had races ran on it
tracksoups = trackinfodf['track_html']
yrmonthdicts_by_track = []
for soup in tracksoups:
    try:
        soup = soup.body.center.findAll('table')[1].tr.findAll('td')
    except:
        soup = soup.body.center.findAll('table')[0].tr.findAll('td')
    years = []
    yrmonths_for_track = []
    for col in soup:
        year = col.text.encode('ascii')[:4]
        years.append(year)
        months = [elem.get('href').encode('ascii')[19:21] for elem in col.findAll('a')]
        yrmonths_for_track.append({year: months})

    yrmonthdicts_by_track.append(yrmonths_for_track)
biggie = dict(zip(trackcodes,yrmonthdicts_by_track))

In [142]:
# Scrapes all the days that the track was in operation for eah month, year, and track combo
%%time
tstorage = []
trackday_lists = []
for track in sorted(biggie.keys()):
    print track
    ystorage = []
    for yeardict in biggie[track]:
        mstorage = []
        if (int(yeardict.keys()[0]) > 2013 and track == 'REM') or trackcodes.index(track) > trackcodes.index('REM'):
            for month in yeardict.values()[0]:
                print month
                monthhtml = BeautifulSoup(requests.get('http://racingchannel.com/archives/{}/{}/{}/default.html'\
                                             .format(track,yeardict.keys()[0],month)).text, 'html.parser').body.center
                try:
                    soup = monthhtml.findAll('table')[1]
                except:
                    soup = monthhtml.findAll('table')[0]
                days = [daytag.get('href')[29:31].encode('ascii') for daytag in soup.findAll('a')]
                mo = daytag.get('href')[19:21].encode('ascii')
                yr = daytag.get('href')[14:18].encode('ascii')
                trk = daytag.get('href')[22:25].encode('ascii')
                for day in days:
                    trackday_lists.append([trk, yr, mo, day]) 
                time.sleep(2)
                #print 'ya'
            time.sleep(5)
    
    print len(trackday_lists)
print len(trackday_lists)

AQU
0
ARL
0
AU1
0
AU2
0
AU3
0
BAY
0
BEL
0
BEU
0
CAL
0
CAM
0
CAN
0
CHU
0
COL
0
CTN
0
DDN
0
DEL
0
DPK
0
ELL
0
EMD
0
EVN
0
FGD
0
FLK
0
FLX
0
FMT
0
FTE
0
GB1
0
GB2
0
GLD
0
GOL
0
GUL
0
HAS
0
HAW
0
HOO
0
HPX
0
IND
0
KEN
0
KND
0
LAU
0
LDN
0
LON
0
LOS
0
MDL
0
MON
0
MTR
0
NCF
0
OAK
0
PAR
10
11
12
04
05
10
11
12
04
05
10
11
12
01
02
03
04
05
09
10
11
12
01
02
03
04
05
09
10
11
12
01
02
03
04
05
09
10
11
12
01
02
03
04
05
10
11
12
01
02
03
04
05
09
10
11
12
01
02
03
04
05
10
11
12
01
02
03
04
05
10
11
12
01
02
03
04
05
10
11
12
01
02
03
04
05
10
11
12
01
02
03
04
05
10
11
12
01
02
03
04
05
10
11
12
01
02
03
04
05
10
11
12
01
02
03
04
05
10
11
12
01
02
03
04
05
10
11
03
04
05
10
11
2379
PEN
2379
PHI
2379
PIM
2379
POR
2379
PRM
2379
REM
03
04
05
06
08
09
10
11
12
03
04
05
08
09
10
11
2605
RET
08
09
10
07
08
09
10
08
09
10
05
06
08
09
10
08
09
10
08
09
10
07
08
09
04
05
06
08
09
10
04
05
06
08
09
10
11
05
06
09
10
11
04
08
09
10
11
12
01
02
08
09
10
11
09
10
07
08
09
10
11
06
07
08
10
11
12
06
07
08


In [204]:
# Was used to deal with two separate runs of the above, and making sure the main 

# print len(trackday_lists)
# trackday_lists2 = trackday_lists
# daydf2 = pd.DataFrame(trackday_lists2, columns=['track','year','month','day'])
# daydf2.to_csv('tempdata/daysscraped2.csv')
# daydf = pd.DataFrame(trackday_lists, columns=['track','year','month','day'])
# daydf.to_csv('tempdata/daysscraped.csv')
# del trackday_lists2, daydf2, trackday_lists, daydf

# daydf = pd.read_csv('tempdata/old/daysscraped.csv')
# daydf2 = pd.read_csv('tempdata/old/daysscraped2.csv')
# daydf.drop(daydf.columns[0], axis=1, inplace=True)
# daydf2.drop(daydf2.columns[0], axis=1, inplace=True)
# bigdaydf = pd.concat([daydf, daydf2])
# bigdaydf.to_csv('tempdata/alldays.csv')
# bigdaydf.head(10)

In [7]:
bigdaydf = pd.read_csv('tempdata/alldays.csv')
bigdaydf = bigdaydf[bigdaydf.columns[-4:]]
bigdaydf.head()

,track,year,month,day
0,AQU,1998,10,28
1,AQU,1998,10,29
2,AQU,1998,10,30
3,AQU,1998,10,31
4,AQU,1998,11,1


In [16]:
print tracks
bigdaydf.track.unique()

['Aqueduct', 'Arlington Park', 'Australian Racing', 'Australian Racing - B', 'Australian Racing - C', 'Bay Meadows', 'Belmont Park', 'Beulah Park', 'Calder Race Course', 'Camarero', 'Canterbury Park', 'Charles Town', 'Churchill Downs', 'Colonial Downs', 'Delta Downs', 'Del Mar', 'Delaware Park', 'Ellis Park', 'Emerald Downs', 'Evangeline Downs', 'Fair Grounds', 'Fairmount Park', 'Fairplex', 'Finger Lakes', 'Fort Erie', 'Golden Gate', 'Great Britain 1', 'Great Britain 2', 'Great Lakes Downs', 'Gulfstream Park', 'Hastings Park', 'Hawthorne Park', 'Hollywood Park', 'Hoosier Park', 'Indiana Downs\n', 'Keeneland', 'Kentucky Downs', 'Laurel Park', 'Louisiana Downs', 'Lone Star Park', 'Los Alamitos', 'Meadowlands', 'Monmouth Park', 'Mountaineer Park', 'Northern California Fairs', 'Oaklawn Park', 'Penn National', 'Philadelphia Park', 'Pimlico', 'Portland Meadows ', 'Prairie Meadows', 'Remington Park', 'Retama Park', 'River Downs', 'Sam Houston', 'Santa Anita', 'Saratoga', 'South Africa1', 'Sou

array(['AQU', 'ARL', 'AU1', 'AU2', 'BAY', 'BEL', 'BEU', 'CAL', 'CAN',
       'CHU', 'COL', 'CTN', 'DDN', 'DEL', 'DPK', 'ELL', 'EMD', 'EVN',
       'FGD', 'FLK', 'FLX', 'FMT', 'FTE', 'GB1', 'GLD', 'GOL', 'GUL',
       'HAS', 'HAW', 'HOO', 'HPX', 'IND', 'KEN', 'KND', 'LAU', 'LDN',
       'LON', 'LOS', 'MDL', 'MON', 'MTR', 'NCF', 'OAK', 'PAR', 'PEN',
       'PHI', 'PIM', 'PRM', 'REM', 'RET', 'RIV', 'SAM', 'SAN', 'SGA',
       'SPT', 'SUF', 'TAM', 'THI', 'TIM', 'TUR', 'WOB', 'ZA1', 'ZA2'], dtype=object)

In [27]:
bigdaydf = bigdaydf.set_index(bigdaydf['track'])
usadaydf = bigdaydf.drop(['GB1','ZA1','ZA2','AU1','AU2','FTE','HAS','NCF','WOB'])
print usadaydf.shape
print bigdaydf.shape
usadaydf.to_csv('tempdata/usadays.csv')

(83873, 4)
(103170, 4)


In [34]:
usadaydf = pd.read_csv('tempdata/usadays.csv')
usadaydf.drop('track.1',axis=1, inplace=True)
usadaydf.head() #<--this is the one we want!!!!

,track,year,month,day
0,AQU,1998,10,28
1,AQU,1998,10,29
2,AQU,1998,10,30
3,AQU,1998,10,31
4,AQU,1998,11,1


In [35]:
def extract(table):
    rows = table.findAll('tr')
    racenum = rows[0].findAll('td')[1].text
    if len(racenum) > 2:
        return 'faketable'
    print table
    
    return [racenum,]
    

In [3]:
##### RUN THESE EXACT LINES GEORGE
%time
usadaydf = pd.read_csv('tempdata/usadays.csv')
usadaydf.drop('track.1',axis=1, inplace=True)
#usadaydf.head() #<--this is the one we want!!!! use this to check too
print usadaydf.shape # this should be like 83000
exceptions = []
superout = []
acc = 0
livee = True
live1 = True
live2 = True
live3 = True
live4 = True
live5 = True
live6 = True
live7 = True
live8 = True
live9 = True
for tup in usadaydf.itertuples():
    time.sleep(0.8)
    magiclink = 'https://www.racingchannel.com/archives/{}/{}/{}/{}{}{}{}.HTM'\
        .format(tup[1],tup[2],tup[3],tup[1],str(tup[2])[2:],tup[3],tup[4])
    try:
        racetest = requests.get(magiclink).text
        soup = BeautifulSoup(racetest, 'html.parser').html.body
        meat = soup.findAll('table')
        superout.append([tup[1],tup[2],tup[3],tup[4],meat])
    except:
        exceptions.append(tup[1],tup[2],tup[3],tup[4],meat)
    acc += 1
    if acc > 100 and livee:
        livee = False
        print "Made it {}".format(acc)
    elif acc > 10000 and live1:
        live1 = False
        print "Made it {}".format(acc)
    elif acc > 20000 and live2:
        live2 = False
        print "Made it {}".format(acc)
    elif acc > 30000 and live3:
        live3 = False
        print "Made it {}".format(acc)
    elif acc > 40000 and live4:
        live4 = False
        print "Made it {}".format(acc)
    elif acc > 50000 and live5:
        live5 = False
        print "Made it {}".format(acc)
    elif acc > 60000 and live6:
        live6 = False
        print "Made it {}".format(acc)
    elif acc > 70000 and live7:
        live7 = False
        print "Made it {}".format(acc)
    elif acc > 80000 and live8:
        live8 = False
        print "Made it {}".format(acc)
        print "Home stretch!!!"  
    

superdf = pd.DataFrame(superout)
superdf.to_csv('tempdata/superdf.csv')
excepdf = pd.DataFrame(exceptions)
excepdf.to_csv('tempdata/excepdf.csv')
superdf.head()
    
    
    

#     for table in meat:
#         try:
#             extract(table)
#         except:
#             exceptions.append([table,tup])
#         break
#     print exceptions
#     break

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 3.81 µs
(83873, 4)CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 3.81 µs
(83873, 4)CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 3.81 µs
(83873, 4)
Made it 101
Made it 101
Made it 101
Made it 10001
Made it 10001
Made it 10001
Made it 20001
Made it 20001
Made it 20001
Made it 30001
Made it 30001
Made it 30001
Made it 40001
Made it 40001
Made it 40001
Made it 50001
Made it 50001
Made it 50001
Made it 60001
Made it 60001
Made it 60001
Made it 70001
Made it 70001
Made it 70001
Made it 80001
Made it 80001
Made it 80001
Home stretch!!!

Home stretch!!!


,0,1,2,3,4
0,AQU,1998,10,28,"[<table bgcolor=""FF,FF,FF"" border=""0""> <tr> <t..."
1,AQU,1998,10,29,"[<table bgcolor=""FF,FF,FF"" border=""0""> <tr> <t..."
2,AQU,1998,10,30,"[<table bgcolor=""FF,FF,FF"" border=""0""> <tr> <t..."
3,AQU,1998,10,31,"[<table bgcolor=""FF,FF,FF"" border=""0""> <tr> <t..."
4,AQU,1998,11,1,[]


,0,1,2,3,4
0,AQU,1998,10,28,"[<table bgcolor=""FF,FF,FF"" border=""0""> <tr> <t..."
1,AQU,1998,10,29,"[<table bgcolor=""FF,FF,FF"" border=""0""> <tr> <t..."
2,AQU,1998,10,30,"[<table bgcolor=""FF,FF,FF"" border=""0""> <tr> <t..."
3,AQU,1998,10,31,"[<table bgcolor=""FF,FF,FF"" border=""0""> <tr> <t..."
4,AQU,1998,11,1,[]



Home stretch!!!


,0,1,2,3,4
0,AQU,1998,10,28,"[<table bgcolor=""FF,FF,FF"" border=""0""> <tr> <t..."
1,AQU,1998,10,29,"[<table bgcolor=""FF,FF,FF"" border=""0""> <tr> <t..."
2,AQU,1998,10,30,"[<table bgcolor=""FF,FF,FF"" border=""0""> <tr> <t..."
3,AQU,1998,10,31,"[<table bgcolor=""FF,FF,FF"" border=""0""> <tr> <t..."
4,AQU,1998,11,1,[]


In [197]:
# tables = soup.findAll('tr')
# print len(tables)
# for elem in tables[:6]:
#     print elem

In [ ]:
# 1) first populate "inlsts" with all the trackkeys, years, months, and days
# 2) scrape each page for its html
# 3) rip through the html for the info we want
# 4) format the info into what we're feeling

In [171]:

superdf = pd.read_csv("~/Desktop/superdf.csv")
superdf = superdf[superdf.columns[-5:]]

In [172]:
def stringtolist(string):
    elems = []
    curr = ''
    acc = 0
    for char in list(string):
        if char in '[':
            pass
        elif (char == ',' and list(string)[acc-1] in '>') or (char == ']'):
            elems.append(curr)
            curr = ''
        else:
            curr += char
        acc += 1
    return elems
a = '[<new>asdf</new>,<yeet>asdfaghfds,FF<yeet>,<FF>masdfasdf</FF>]'
print stringtolist(a)

['<new>asdf</new>', '<yeet>asdfaghfds,FF<yeet>', '<FF>masdfasdf</FF>']


In [173]:
errortups = []


count = 0
allthethings = []
for tup in superdf.itertuples():
    count += 1
    if count > 20:
        break
    elif tup[-1] == '[]':
        errortups.append(tup[:4])
    else:
        tables = [BeautifulSoup(elem).html.body.table for elem in stringtolist(tup[-1])]
        infos1 = []
        infos2 = []
        infos3 = []
        othercrap = []
        others3 = []
        for table in tables:
            boldys = []
            numsiwant = []
            for td in table.findAll('td'):
                bolds = td.findAll('b')
                if len(bolds) > 0:
                    boldys.append(bolds[0].text.encode('ascii'))
                if td.get('align') == 'RIGHT' and '<b>' not in str(td):
                    content = td.text.encode('ascii')
                    if content != '':
                        numsiwant.append(content)
            infopack3 = numsiwant[:6]
            infos3.append(infopack3)
            other3 = numsiwant[6:]
            others3.append(other3)
            try:
                infopack1 = [boldys[1],boldys[6],boldys[7],boldys[9],boldys[10],boldys[12],boldys[13]]
                infopack2 = boldys[14:]
                infos1.append(infopack1)
                infos2.append(infopack2)
            except:
                #print "index error %i" % len(infopack1)
                othercrap.append((boldys,table))
    allthethings.append([infos1,infos2,infos3,others3,othercrap])

#         print tup[:4]
#         for i in range(len(infos1)):
#             print infos1[i]
#             print infos2[i]
#             print infos3[i]
#         print '\n'
#         print others3
#         print othercrap
#         print '\n\n\n'

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

class RaceDataSet(object):
    
    def __init__(self, num):
        self.num = num
        self.first = None
        self.second = None
        self.third = None
        self.firstnum = None
        self.secondnum = None
        self.thirdnum = None
        self.win = None
        self.place1 = None
        self.place2 = None
        self.show1 = None
        self.show2 = None
        self.show3 = None
        self.otherstuff = []

alldays = []
for day in allthethings:
    races = []
    for i in range(len(day[0])):
        infopack = day[0][i]
        #print infopack
        racedata = RaceDataSet(infopack[0])
        racedata.firstnum = infopack[1]
        racedata.first = infopack[2]
        racedata.secondnum = infopack[3]
        racedata.second = infopack[4]
        racedata.thirdnum = infopack[5]
        racedata.third = infopack[6]
        races.append(racedata)
    alldays.append(races)
for day in alldays:
    print "----"
    for race in day:
        print race.first
            


----
CURVY IMAGE
MILLIONDOLLARSMILE
ROOK
WAPO
FUNNY TOY (ARG)
RUBY RUBLES
EXCHANGE PLACE
EARLY WARNING
JET BLACK
----
CRITICS ACCLAIM
STORM IMP
BETTER THAN HONOUR
BUNDLING
SANFORD SAM
LE MISTRAL
PHONE THE KING
BRAC DRIFTER
HANDEL
----
DR. CHABOT
MAWHUB
LATE CARSON
BIG NEWS
RETIRING DON
COMIC PROSPECT
SOLAR COLONY
WILDFAREL
HIP HAPPY HIP
----
SILVER KIPPER

CAJUN SEASON
CIGAR CHARLIE
CAYMAN CAT


SNIT

CARROM
----
SILVER KIPPER

CAJUN SEASON
CIGAR CHARLIE
CAYMAN CAT


SNIT

CARROM
----
SILVER KIPPER

CAJUN SEASON
CIGAR CHARLIE
CAYMAN CAT


SNIT

CARROM
----
SILVER KIPPER

CAJUN SEASON
CIGAR CHARLIE
CAYMAN CAT


SNIT

CARROM
----
SILVER KIPPER

CAJUN SEASON
CIGAR CHARLIE
CAYMAN CAT


SNIT

CARROM
----
SILVER KIPPER

CAJUN SEASON
CIGAR CHARLIE
CAYMAN CAT


SNIT

CARROM
----
SILVER KIPPER

CAJUN SEASON
CIGAR CHARLIE
CAYMAN CAT


SNIT

CARROM
----
CASCADE HALO
GOLD 'N SUGAR
MS. CALABASH
DREWS TOUCH
GOLD CERTIFICATE
SIMUD (CHI)
COUNT ON STEVE
WOULDN'T WE ALL
MR. SINATRA

----
DOUBLE CONQUEST